In [1]:
import librosa
import librosa.display
import IPython.display as ipd
import os
import numpy as np
import itertools
import pandas as pd
import math
from scipy.signal import get_window
import scipy
from sklearn import preprocessing
from scipy.io.wavfile import read
import python_speech_features as mfcc
from sklearn.mixture import GaussianMixture 
import glob
import pickle
import joblib
from sklearn.metrics import f1_score
import soundfile

In [2]:
def calculate_delta(array):
    rows,cols = array.shape
    deltas = np.zeros((rows,20))
    N = 2
    for i in range(rows):
        index = []
        j = 1
        while j <= N:
            if i-j < 0:
                first =0
            else:
                first = i-j
            if i+j > rows-1:
                second = rows-1
            else:
                second = i+j 
            index.append((second,first))
            j+=1
        deltas[i] = ( array[index[0][0]]-array[index[0][1]] + (2 * (array[index[1][0]]-array[index[1][1]])) ) / 10
    return deltas

# Feature Extraction for both train and test data

def extract_features_for_data(audio,sr):
    mfcc_feature = mfcc.mfcc(audio,sr, 0.025, 0.01,20, nfft = 1200 ,appendEnergy=True)
    mfcc_feature = preprocessing.scale(mfcc_feature)
    delta = calculate_delta(mfcc_feature)
    combined = np.hstack((mfcc_feature,delta)) 
    return combined




In [3]:
###

featuresOpen = np.asarray(())
directory = './Open/'
for audio in os.listdir('./Open/'):
    audio_path = directory + audio
    sr,audio = read(audio_path)
    vector1   = extract_features_for_data(audio,sr)
    if featuresOpen.size == 0:
        featuresOpen = vector1
    else:
        featuresOpen=np.vstack((featuresOpen, vector1 ))

featuresClose = np.asarray(())
directory = './Close/'
for audio in os.listdir('./Close/'):
    audio_path = directory + audio
    sr,audio = read(audio_path)
    vector1   = extract_features_for_data(audio,sr)
    if featuresClose.size == 0:
        featuresClose = vector1
    else:
        featuresClose=np.vstack((featuresClose, vector1 ))

featuresPush = np.asarray(())
directory = './Push/'
for audio in os.listdir('./Push/'):
    audio_path = directory + audio
    sr,audio = read(audio_path)
    vector1   = extract_features_for_data(audio,sr)
    if featuresPush.size == 0:
        featuresPush = vector1
    else:
        featuresPush=np.vstack((featuresPush, vector1 ))

featuresPull = np.asarray(())
directory = './Pull/'
for audio in os.listdir('./Pull/'):
    audio_path = directory + audio
    sr,audio = read(audio_path)
    vector1   = extract_features_for_data(audio,sr)
    if featuresPull.size == 0:
        featuresPull = vector1
    else:
        featuresPull=np.vstack((featuresPull, vector1 ))


In [4]:
Open_gmm = GaussianMixture(n_components=  6, max_iter = 200, covariance_type='diag',n_init =3)
Open_gmm.fit(featuresOpen)

Close_gmm = GaussianMixture(n_components= 6, max_iter = 200, covariance_type='diag',n_init =3)
Close_gmm.fit(featuresClose)

Push_gmm = GaussianMixture(n_components= 6, max_iter = 200, covariance_type='diag',n_init =3)
Push_gmm.fit(featuresPush)

Pull_gmm = GaussianMixture(n_components= 6, max_iter = 200, covariance_type='diag',n_init =3)
Pull_gmm.fit(featuresPull)

GaussianMixture(covariance_type='diag', max_iter=200, n_components=6, n_init=3)

In [5]:
pickle.dump(Open_gmm,open('Open.gmm','wb'))
pickle.dump(Close_gmm,open('Close.gmm','wb'))
pickle.dump(Push_gmm,open('Push.gmm','wb'))
pickle.dump(Pull_gmm,open('Pull.gmm','wb'))

In [6]:
lstFolders1 = ['Open', 'Close', 'Push','Pull']
models = [joblib.load(f + '.gmm') for f in lstFolders1]

In [7]:
models

[GaussianMixture(covariance_type='diag', max_iter=200, n_components=6, n_init=3),
 GaussianMixture(covariance_type='diag', max_iter=200, n_components=6, n_init=3),
 GaussianMixture(covariance_type='diag', max_iter=200, n_components=6, n_init=3),
 GaussianMixture(covariance_type='diag', max_iter=200, n_components=6, n_init=3)]

In [8]:
y1 = []
y1_true = []
lstFolders1 = ['Open', 'Close', 'Push','Pull']
for folder in lstFolders1:
    # try:
        lstFiles = glob.glob('./{}/*'.format(folder))
        for f in lstFiles:
            sr,audio = read(audio_path)
            vstack = extract_features_for_data(audio, sr)
            output = []
            for model in models:
                scores = np.array(model.score(vstack))
                output.append(scores.sum())

            winner1 = np.argmax(output)
            y1_true.append(winner1)
            y1.append(lstFolders1.index(folder))
    

In [9]:
f1_score(y1, y1_true, average='micro')

0.25

In [10]:
sr,audio = read('./Open/Open(5).wav')
test2 = extract_features_for_data(audio, sr)
sr,audio = read('./Close/Close(5).wav')
test3= extract_features_for_data(audio, sr)
sr,audio = read('./Push/Push(5).wav')
test4 = extract_features_for_data(audio, sr)
sr,audio = read('./Pull/Pull(5).wav')
test5 = extract_features_for_data(audio, sr)

scores_1 = np.array(Open_gmm.score(test2))
scores_2 = np.array(Close_gmm.score(test3))
scores_3 = np.array(Push_gmm.score(test4))
scores_4 = np.array(Push_gmm.score(test5))

print(scores_1)
print(scores_2)
print(scores_3)
print(scores_4)

-17.57812744493198
-20.812400702056475
-21.322272808478168
-21.202818955772017
